# 🚀 Integração Notebooks ↔ Produção

## Solução para Workflow Eficiente: Notebooks que Salvam Modelos para Produção

<div align="center">

```
┌─────────────────────────────────────────────────────────────┐
│  NOTEBOOK → PRODUÇÃO SEM RETRAINING INEFICIENTE           │
└─────────────────────────────────────────────────────────────┘
```

![Workflow](https://img.shields.io/badge/Workflow-Optimized-green)
![Efficiency](https://img.shields.io/badge/Efficiency-Maximized-blue)
![Integration](https://img.shields.io/badge/Integration-Seamless-success)

</div>

---

### 🎯 **PROBLEMA IDENTIFICADO**

Atualmente, o workflow força **duplicação de esforço**:
1. **Notebooks**: Experimentam e descobrem melhores modelos
2. **train.py**: Retreina os mesmos modelos para produção

### ✅ **SOLUÇÃO IMPLEMENTADA**

Este notebook demonstra como **notebooks podem salvar modelos diretamente para produção**, eliminando retraining desnecessário.

---

### 📋 **WORKFLOW OTIMIZADO**

```
Notebook Experimentation → Model Training → Direct Save → Production
                                      ↓
                             Sem retraining no train.py!
```

---

In [1]:
# ============================================================================
# SETUP: IMPORTS E CONFIGURAÇÃO
# ============================================================================

import sys
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

# Add project root to path
project_root = Path.cwd().parent
if str(project_root) not in sys.path:
    sys.path.append(str(project_root))

# Import project utilities
from utils.data import load_artifact, save_trained_model_for_production
from utils.modeling import FraudMetrics
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, roc_auc_score

print("✅ Setup completo!")
print(f"📁 Project root: {project_root}")

ModuleNotFoundError: No module named 'utils'

In [ ]:
# ============================================================================
# CARREGAR DADOS PREPARADOS
# ============================================================================

print("📊 Carregando dados preparados...")

# Carregar dados preparados pelo notebook de feature engineering
try:
    X_train = load_artifact(project_root / "artifacts" / "X_train_temporal_clean.parquet")
    y_train = load_artifact(project_root / "artifacts" / "y_train_processed.parquet")['target']
    X_test = load_artifact(project_root / "artifacts" / "X_test_temporal_clean.parquet")
    y_test = load_artifact(project_root / "artifacts" / "y_test_processed.parquet")['target']

    print("✅ Dados carregados com sucesso!")
    print(f"📈 Train: {X_train.shape[0]:,} amostras, {X_train.shape[1]} features")
    print(f"🧪 Test: {X_test.shape[0]:,} amostras, {X_test.shape[1]} features")
    print(".1%")

except Exception as e:
    print(f"❌ Erro carregando dados: {e}")
    print("Execute primeiro o notebook 01_feature_engineering.ipynb")
    exit()

In [ ]:
# ============================================================================
# TREINAMENTO DE MODELOS NO NOTEBOOK
# ============================================================================

print("🤖 Iniciando treinamento de modelos...")

# Definir modelos para testar
models = {
    'RandomForest': RandomForestClassifier(
        n_estimators=100,
        max_depth=10,
        random_state=42,
        n_jobs=-1
    ),
    'GradientBoosting': GradientBoostingClassifier(
        n_estimators=100,
        learning_rate=0.1,
        max_depth=5,
        random_state=42
    )
}

# Treinar e avaliar modelos
model_results = {}

for name, model in models.items():
    print(f"\n🏋️ Treinando {name}...")

    # Treinar modelo
    model.fit(X_train, y_train)

    # Avaliar no conjunto de teste
    y_pred = model.predict(X_test)
    y_pred_proba = model.predict_proba(X_test)[:, 1]

    # Calcular métricas
    roc_auc = roc_auc_score(y_test, y_pred_proba)

    # Cross-validation para robustez
    cv_scores = cross_val_score(model, X_train, y_train, cv=3, scoring='roc_auc')

    model_results[name] = {
        'model': model,
        'roc_auc_test': roc_auc,
        'cv_mean': cv_scores.mean(),
        'cv_std': cv_scores.std(),
        'classification_report': classification_report(y_test, y_pred, output_dict=True)
    }

    print(".4f"    print(".4f"    print(".4f"
# Selecionar melhor modelo
best_model_name = max(model_results.keys(), key=lambda x: model_results[x]['roc_auc_test'])
best_model = model_results[best_model_name]['model']
best_score = model_results[best_model_name]['roc_auc_test']

print(f"\n🏆 Melhor modelo: {best_model_name} (ROC-AUC: {best_score:.4f})")

In [ ]:
# ============================================================================
# SALVAR MODELO DIRETAMENTE PARA PRODUÇÃO (SEM RETRAINING!)
# ============================================================================

print("💾 Salvando modelo treinado diretamente para produção...")

# Metadados do modelo
model_metadata = {
    'training_date': datetime.now().isoformat(),
    'notebook_version': 'integrated_workflow_v1.0',
    'data_version': 'temporal_clean_v2.0',
    'hyperparameters': {
        'n_estimators': getattr(best_model, 'n_estimators', None),
        'max_depth': getattr(best_model, 'max_depth', None),
        'learning_rate': getattr(best_model, 'learning_rate', None),
        'random_state': getattr(best_model, 'random_state', None)
    },
    'performance': {
        'roc_auc_test': model_results[best_model_name]['roc_auc_test'],
        'cv_mean': model_results[best_model_name]['cv_mean'],
        'cv_std': model_results[best_model_name]['cv_std']
    },
    'features_used': list(X_train.columns),
    'target_distribution': y_train.value_counts().to_dict()
}

# Salvar modelo usando a nova função
save_result = save_trained_model_for_production(
    model=best_model,
    model_name=f"fraud_detection_{best_model_name.lower()}",
    experiment_id=f"notebook_integration_{datetime.now().strftime('%Y%m%d')}",
    metadata=model_metadata
)

print("
✅ Modelo salvo com sucesso!"print(f"📁 Caminho: {save_result['model_path']}")
print(f"📄 Metadata: {save_result['metadata_path']}")
print(f"🧪 Experiment ID: {save_result['experiment_id']}")

In [ ]:
# ============================================================================
# PROMOVER PARA PRODUÇÃO
# ============================================================================

print("🚀 Promovendo modelo para produção...")

# Comando para promover o modelo (usando o novo fluxo centralizado)
promotion_command = save_result['promotion_command']

print("Execute o seguinte comando no terminal para promover:")
print(f"🔧 {promotion_command}")
print()
print("Ou liste modelos disponíveis primeiro:")
print(f"🔧 python src/modeling/train.py --list_notebook_models")

In [ ]:
# ============================================================================
# VERIFICAÇÃO: CARREGAR E TESTAR MODELO SALVO
# ============================================================================

print("🔍 Verificando modelo salvo...")

import pickle
import json

# Carregar modelo salvo
with open(save_result['model_path'], 'rb') as f:
    loaded_model = pickle.load(f)

# Carregar metadata
with open(save_result['metadata_path'], 'r') as f:
    loaded_metadata = json.load(f)

print("✅ Modelo carregado com sucesso!")
print(f"📊 Tipo: {type(loaded_model).__name__}")
print(f"🎯 ROC-AUC salvo: {loaded_metadata['performance']['roc_auc_test']:.4f}")

# Testar predições
test_predictions = loaded_model.predict(X_test[:10])
test_probabilities = loaded_model.predict_proba(X_test[:10])

print(f"🧪 Teste de predição: {len(test_predictions)} amostras")
print(f"📈 Probabilidades shape: {test_probabilities.shape}")

print("\n✅ Verificação completa - modelo pronto para produção!")

# 🎉 **WORKFLOW OTIMIZADO CONCLUÍDO!**

## 📊 **RESUMO DA SOLUÇÃO**

### ❌ **Antes (Ineficiente)**
```
Notebook Experimentation → Descobrir Melhor Modelo
                                      ↓
train.py → Retreinar Mesmo Modelo → Produção
```

### ✅ **Agora (Otimizado)**
```
Notebook Experimentation → Treinar Modelo → Salvar Direto → Produção
                                      ↓
                         Sem retraining desnecessário!
```

---

## 🔧 **COMO USAR EM SEUS NOTEBOOKS**

### **1. Após treinar seu modelo:**
```python
from utils.data import save_trained_model_for_production

# Seu modelo treinado
best_model = ...

# Salvar diretamente para produção
save_result = save_trained_model_for_production(
    model=best_model,
    model_name="meu_modelo_fraud",
    experiment_id="exp_001",
    metadata={'roc_auc': 0.95, 'features': list(X_train.columns)}
)
```

### **2. Promover para produção:**
```bash
python src/modeling/train.py --promote_trained artifacts/models/meu_modelo_fraud.pkl
```

---

## 🚀 **PRÓXIMOS PASSOS**

1. **Integre em seus notebooks** de modelagem
2. **Use quando quiser pular retraining** desnecessário
3. **Mantenha train.py** para casos onde retraining é necessário
4. **Documente** qual abordagem usar em cada cenário

---

## 📈 **BENEFÍCIOS**

- ⚡ **Velocidade**: Sem retraining desnecessário
- 🎯 **Precisão**: Mesmo modelo, mesmas métricas
- 🔄 **Flexibilidade**: Escolha entre notebook direto ou train.py
- 📊 **Rastreabilidade**: Metadata completa preservada
- 🛡️ **Produção**: Modelos prontos para deployment imediato

**Fim do workflow otimizado! 🎉**